# Twitter Data Analysis

In [1]:
from twython import Twython
import time
import math

def twitter_oauth_login():
    API_KEY = '0PXGjABM0MBvuIorEJF37MjtR'
    API_SECRET = 'NVkG6W75xGAlgCa6dMg1vmH1K8iZOLIuc9sqFoR7iLwKukAnkD'
    ACCESS_TOKEN = '1548649856-wQnlcp8FgrtbmMLdTTBBfVP9EkM5KUIyoswjrxU'
    ACCESS_TOKEN_SECRET = 'dBRn8dYIrsNCDsnvw4ubnvKtv6av9Ua68GL3PeayRFLmg'
    twitter = Twython(API_KEY,API_SECRET,ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
    return twitter

In [2]:
twitter = twitter_oauth_login()
friends_ids = twitter.get_friends_ids(count=5000)
friends_ids = friends_ids['ids']
followers_ids = twitter.get_followers_ids(count=5000)
followers_ids = followers_ids['ids']

In [3]:
friends_set = set(friends_ids)
followers_set = set(followers_ids)

print('Number of Twitter users who either are our friend or follow you:')
print(len(friends_set.union(followers_set)))

print('Number of Twitter users who follow you and are your friend:')
print(len(friends_set & followers_set))

print('Number of Twitter users you follow that don\'t follow you:')
print(len(friends_set - followers_set))

print('Number of Twitter users who follow you that you don\'t follow:')
print(len(followers_set - friends_set))

Number of Twitter users who either are our friend or follow you:
28
Number of Twitter users who follow you and are your friend:
2
Number of Twitter users you follow that don't follow you:
26
Number of Twitter users who follow you that you don't follow:
0


In [4]:
def pull_users_profiles(ids):
    users = []
    for i in range(0,len(ids),100):
        batch = ids[i:i+100]
        users += twitter.lookup_user(user_id=batch)
    return users

In [5]:
def pull_users_profiles_limit_aware(ids):
    users = []
    start_time = time.time()
    # Must look up users in
    for i in range(0, len(ids), 10):
        batch = ids[i:i + 10]
        users += twitter.lookup_user(user_id=batch)
        calls_left = float(twitter.get_lastfunction_header('X-Rate-Limit-Remaining'))
        time_remaining_in_window = rate_limit_window - (time.time()-start_time)
        sleep_duration = math.ceil(time_remaining_in_window/calls_left)
        print('Sleeping for: ' + str(sleep_duration) + ' seconds; ' +
        str(calls_left) + ' API calls remaining')
        time.sleep(sleep_duration)
    return users

In [6]:
friends_profiles = pull_users_profiles(friends_ids)
followers_profiles = pull_users_profiles(followers_ids)

friends_screen_names = [p.get('screen_name') for p in friends_profiles]
print(friends_screen_names)

['Tesla', 'BarackObama', 'cyyy0411', 'phebeshi', 'invincible_Dong', 'Xiao_CharlieXC', 'RainieLiuuu', 'NASA', 'Google', 'JINZANG999', 'BravoZhuang', 'Dongqi_Fu', 'MrAncelotti', 'officialpepe', 'RealMadridWorld', 'MarceloM12', 'Real_Madrid__CF', 'G_Higuain', 'MesutOzil1088', 'lafabricacrm', 'aarbeloa17', 'SergioRamos', 'Cristiano', 'RealPotterFans', 'EmmaWatson', 'britneyspears', 'KingJames', 'taylorswift13']


In [10]:
import json

def save_json(filename,data):
    with open(filename,"w", encoding="utf8") as outfile:
        json.dump(data,outfile)
        

def load_json(filename):
    with open(filename) as infile:
        data = json.load(infile)
    return data

In [11]:
#Test on json script
fname = 'test_friends_profile.json'
save_json(fname, friends_profiles)
test_load = load_json(fname)
print(test_load[0])

{'id': 13298072, 'id_str': '13298072', 'name': 'Tesla', 'screen_name': 'Tesla', 'location': '', 'description': 'Electric cars, giant batteries and solar', 'url': 'https://t.co/MxFwoUXrrG', 'entities': {'url': {'urls': [{'url': 'https://t.co/MxFwoUXrrG', 'expanded_url': 'http://ts.la/w0i', 'display_url': 'ts.la/w0i', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 2325631, 'friends_count': 128, 'listed_count': 11295, 'created_at': 'Sun Feb 10 01:12:32 +0000 2008', 'favourites_count': 1784, 'utc_offset': -28800, 'time_zone': 'Pacific Time (US & Canada)', 'geo_enabled': True, 'verified': True, 'statuses_count': 6441, 'lang': 'en', 'status': {'created_at': 'Thu Feb 01 03:51:49 +0000 2018', 'id': 958910735567826944, 'id_str': '958910735567826944', 'text': '.@MKBHD reviews Model 3 https://t.co/kA4Gs5O0Ps', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'MKBHD', 'name': 'Marques Brownlee', 'id': 298

In [12]:
import pymongo

host_string = "mongodb://localhost"
port = 27017
mongo_client = pymongo.MongoClient(host_string, port)

#get a reference to the mongo database 'test'
mongo_db = mongo_client['test']

#get a reference to the 'user profiles' collection in the 'test' database
user_profiles_collection = mongo_db['user_profiles']

user_profiles_collection.insert(friends_profiles)
user_profiles_collection.insert(followers_profiles)

/Users/zhangpeng/venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  del sys.path[0]
/Users/zhangpeng/venv/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5a7663f1945ee64bc9a908cd'), ObjectId('5a7663f1945ee64bc9a908ce')]

In [13]:
def save_json_to_mongo(data, mongo_db,mongo_db_collection,host_string="localhost",port=27017):
    mongo_client = pymongo.MongoClient(host_string,port)
    mongo_db = mongo_client[mongo_db]
    collection  = mongo_db[mongo_db_collection]
    inserted_object_ids = collection.insert(data)
    return inserted_object_ids

In [15]:
fname = 'test_friends_profile.json'
load_json(fname)

[{'contributors_enabled': False,
  'created_at': 'Sun Feb 10 01:12:32 +0000 2008',
  'default_profile': False,
  'default_profile_image': False,
  'description': 'Electric cars, giant batteries and solar',
  'entities': {'description': {'urls': []},
   'url': {'urls': [{'display_url': 'ts.la/w0i',
      'expanded_url': 'http://ts.la/w0i',
      'indices': [0, 23],
      'url': 'https://t.co/MxFwoUXrrG'}]}},
  'favourites_count': 1784,
  'follow_request_sent': False,
  'followers_count': 2325631,
  'following': True,
  'friends_count': 128,
  'geo_enabled': True,
  'has_extended_profile': False,
  'id': 13298072,
  'id_str': '13298072',
  'is_translation_enabled': False,
  'is_translator': False,
  'lang': 'en',
  'listed_count': 11295,
  'location': '',
  'name': 'Tesla',
  'notifications': False,
  'profile_background_color': '666666',
  'profile_background_image_url': 'http://pbs.twimg.com/profile_background_images/812177545/c69f76a0142059d01f86203029dbaf6e.png',
  'profile_backgroun

In [17]:
geo_enabled = [p['geo_enabled'] for p in friends_profiles]
geo_enabled.count(1)

7

In [18]:
location = [p['location'] for p in friends_profiles]
location.count('')

11

In [19]:
print(set(location))

{'', 'Madrid, Spain', 'Torino, Piemonte', 'Hogwarts ', 'Amongst La Familia! ', 'United States', 'Official Profile', 'Manhattan, NY', 'Washington, DC', 'Mountain View, CA', 'Estadio Santiago Bernabeu', 'England', 'Los Angeles, CA', 'California, USA', 'Madrid', 'Madrid, Comunidad de Madrid', 'Madrid, Spain.'}


In [20]:
time_zone = [p['time_zone'] for p in friends_profiles]
time_zone.count(None)

8

In [21]:
print(set(time_zone))

{None, 'Quito', 'London', 'Eastern Time (US & Canada)', 'Mid-Atlantic', 'Athens', 'Caracas', 'Madrid', 'Pacific Time (US & Canada)'}


In [24]:
status_geo = [p['status']['geo'] for p in friends_profiles if('status' in p and p['status']['geo'] is not None)]
if status_geo:
    print(status_geo[0])
print(len(status_geo))

0


In [30]:
status_geo = []
status_geo_screen_names = []
for fp in friends_profiles:
    if('status' in fp and fp['status']['geo'] is not None and 'screen_name' in fp):
        status_geo.append(fp['status']['geo'])
        status_geo_screen_names.append(fp['screen_name'])

In [35]:
import folium


map =folium.Map(location=[48,-102],zoom_start=3)
for sg,sn in zip(status_geo, status_geo_screen_names):
    map.simple_marker(sg['coordinates'],popup=str(sn))
map.save('us_states.html')